<a href="https://colab.research.google.com/github/mtdmo/kagglespot/blob/main/Dec_Playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prep

In [20]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

In [ ]:
!pip install mljar-supervised

In [ ]:
!pip install twilio

In [22]:
import json
with open('/content/drive/MyDrive/Colab Notebooks/Config files/cred.json') as f:
  data = json.load(f)

In [23]:
import pandas as pd
import seaborn as sn
from google.colab import files
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from supervised import AutoML
from google.colab import files

In [24]:
from twilio.rest import Client
client = Client(data['TwilioAccount'], data['TwilioKey'])

# Data Importing

In [25]:
## Pull in Kaggle data
dftest = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Dec Playground/test.csv")

In [26]:
dftrain = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Dec Playground/train.csv")

In [27]:
#print("Test Data Columns:")
#print(dftest.columns)
#print("\n")
#print("Training Data Columns:")
#print(dftrain.columns)

In [28]:
dftrain["Cover_Type"].value_counts()

2    2262087
1    1468136
3     195712
7      62261
6      11426
4        377
5          1
Name: Cover_Type, dtype: int64

In [29]:
#Confirming entire load of CSV file
sum(dftrain["Cover_Type"].value_counts())/4000000

1.0

In [30]:
X_train, X_test, y_train, y_test = train_test_split(
    dftrain[dftrain.columns[:-1]], dftrain["Cover_Type"], test_size=0.25,
)

# Model Generation

In [ ]:
# train models with AutoML
automl = AutoML(mode="Explain", algorithms=["Xgboost"])
automl.fit(X_train, y_train)

# compute the accuracy on test data
predictions = automl.predict_all(X_test)
print(predictions.head())
print("Test accuracy:", accuracy_score(y_test, predictions["label"].astype(int)))
testacc = "Test accuracy:", accuracy_score(y_test, predictions["label"].astype(int))
#Text me the model is completed
try:
  client.messages.create(to="+15733563395", 
                        from_="+13305221059", 
                        body=f"Hi I am done with the model. Model Results: {testacc}")
except:
  print("Error sending text message")
#Zip the AutoML contents and download
try:
  !zip -r /content/file.zip /content/AutoML_1/
  files.download("/content/file.zip")
except:
  print("Error saving AutoML file")

AutoML directory: AutoML_5
The task is multiclass_classification with evaluation metric logloss
AutoML will use algorithms: ['Xgboost']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 1 model


# Submission File Creation

In [ ]:
automl = AutoML(results_path='/content/drive/MyDrive/Colab Notebooks/Dec Playground/AutoML_1')
finprediction = automl.predict(dftest)

In [ ]:
finpredDF = pd.DataFrame(finprediction)

In [ ]:
finpredDF2 = dftest.join(finpredDF)

In [ ]:
finpredDF2.columns = [*finpredDF2.columns[:-1], 'Cover_Type']

In [ ]:
finpredDF2[['Id','Cover_Type']].to_csv("submission1.csv", index=False)